In [1]:
import tensorflow as tf
from mit_bih_data_maker import mit_bih_data_maker

def test_label_maker(label):
    label_map = {"N":0, "L":1, "R":2, "/":3, "V":4, "A":5}
    if label in label_map:
        return label_map[label]
    return "Omit"

data_maker = mit_bih_data_maker()
data_maker.config("../database/mit-bih/", "../database/mit-bih", "../database/mit-bih-r-label")
#rec_list = ["100", "109", "118", "107", "208", "232"]
rec_list = []
data_maker.load_data(rec_list)

sample_num = -1
label_percent = {0:0.197, 1:0.220, 2:0.191, 3:0.183, 4:0.087, 5:0.122}
train_percent = 0.7
ecg_data = data_maker.make_by_r_peak(before_r = 90, after_r = 146, sample_num = sample_num, label_maker = test_label_maker, \
                                     label_percent = label_percent, train_percent = train_percent)
ecg_data.print_summary()

TRAIN Summay
Sample num: 14473, feature num: 236
Sample num for each label:
Label 0: 2851
Label 1: 3190
Label 2: 2767
Label 3: 2621
Label 4: 1270
Label 5: 1774

TEST Summary
Sample num: 6203, feature num: 236
Sample num for each label:
Label 0: 1222
Label 1: 1359
Label 2: 1182
Label 3: 1163
Label 4: 529
Label 5: 748


In [2]:
import pywt

import numpy as np
from statsmodels.tsa.ar_model import AR

db8 = pywt.Wavelet('db8')
fea_train = []
for i in xrange(ecg_data.train.x.shape[0]):
    cA3, cD3, cD2, cD1 = pywt.wavedec(ecg_data.train.x[i].tolist(), db8, level = 3)
    model = AR(ecg_data.train.x[i].tolist())
    result = model.fit(maxlag = 4)
    fea_train.append(ecg_data.train.x[i].tolist() + cA3.tolist() + result.params[1:].tolist())

fea_test = []
for i in xrange(ecg_data.test.x.shape[0]):
    cA3, cD3, cD2, cD1 = pywt.wavedec(ecg_data.test.x[i].tolist(), db8, level = 3)
    model = AR(ecg_data.test.x[i].tolist())
    result = model.fit(maxlag = 4)
    fea_test.append(ecg_data.test.x[i].tolist() + cA3.tolist() + result.params[1:].tolist())
        
print np.array(fea_train).shape
print np.array(fea_test).shape

(14473, 282)
(6203, 282)


In [4]:
import numpy as np

def weight_var(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_var(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 282])
y_ = tf.placeholder(tf.float32, [None])
y_onehot = tf.one_hot(tf.cast(y_, tf.int64), 6, 1.0, 0.0)

#隐层
W_h = weight_var([282, 32])
b_h = bias_var([32])
h = tf.nn.relu(tf.matmul(x, W_h) + b_h)

#输出层
W_o = weight_var([32, 6])
b_o = bias_var([6])
o = tf.nn.softmax(tf.matmul(h, W_o) + b_o)

cross_entropy = -tf.reduce_sum(y_onehot * tf.log(o))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

corr_pred = tf.equal(tf.arg_max(o, 1), tf.arg_max(y_onehot, 1))
accuracy = tf.reduce_mean(tf.cast(corr_pred, tf.float32))

sess.run(tf.initialize_all_variables())

for i in xrange(200000):
    batch = ecg_data.train.next_batch(50)
    
    if i % 1000 == 0:
        train_accuacy = accuracy.eval(feed_dict={x: np.array(fea_test), y_: ecg_data.test.y})
        print("step %d, training accuracy %g"%(i, train_accuacy))
        
    db8 = pywt.Wavelet('db8')
    fea_train = []
    for i in xrange(batch[0].shape[0]):
        cA3, cD3, cD2, cD1 = pywt.wavedec(batch[0][i].tolist(), db8, level = 3)
        model = AR(batch[0][i].tolist())
        result = model.fit(maxlag = 4)
        fea_train.append(batch[0][i].tolist() + cA3.tolist() + result.params[1:].tolist())
        
    sess.run(train_step, feed_dict = {x : np.array(fea_train), y_ : batch[1]})

step 0, training accuracy 0.217959
step 1000, training accuracy 0.82315
step 2000, training accuracy 0.859584
step 3000, training accuracy 0.876189
step 4000, training accuracy 0.8857
step 5000, training accuracy 0.894245
step 6000, training accuracy 0.903273
step 7000, training accuracy 0.907787
step 8000, training accuracy 0.912784
step 9000, training accuracy 0.915686
step 10000, training accuracy 0.919071
step 11000, training accuracy 0.92149
step 12000, training accuracy 0.924553
step 13000, training accuracy 0.927293
step 14000, training accuracy 0.929228
step 15000, training accuracy 0.933581
step 16000, training accuracy 0.933097
step 17000, training accuracy 0.937611
step 18000, training accuracy 0.938417
step 19000, training accuracy 0.939545
step 20000, training accuracy 0.940835
step 21000, training accuracy 0.94277
step 22000, training accuracy 0.945188
step 23000, training accuracy 0.944059
step 24000, training accuracy 0.945833
step 25000, training accuracy 0.945994
step

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x4018ab10>> ignored


KeyboardInterrupt: 